In [1]:
# ! pip install networkx

In [2]:
import networkx as nx 
import pandas as pd 
import matplotlib.pyplot as plt 
import scipy
import numpy as np

In [3]:
# importing data as dataframes
us_bus = pd.read_csv(r'C:\Users\Mani\myjupyter_notes\Truist\.ipynb_checkpoints\user_business.dat', sep='\t', skiprows=0, header=None)
us_us = pd.read_csv(r'C:\Users\Mani\myjupyter_notes\Truist\.ipynb_checkpoints\user_user.dat', sep='\t', skiprows=0, header=None)

In [4]:
# Setting header names
us_us.columns = ['User','Users','relationship'] 
us_bus.columns = ['User','Business','Rating']

In [5]:
us_bus['User'] = 'u' + us_bus['User'].astype(str)
us_bus['Business'] = 'b' + us_bus['Business'].astype(str)
us_bus['Rating'] =  us_bus['Rating'].astype(int)

In [6]:
us_us['User'] = 'u' + us_us['User'].astype(str)
us_us['Users'] = 'u' + us_us['Users'].astype(str) #Assigning us to Users
us_us['relationship']= us_us['relationship'].astype(int)

In [7]:
# adding a dummy user u0, who is connected to user 1 and rated same business
new_row = pd.DataFrame({'User':'u0', 'Business':'b8391', 'Rating':1,},index =[0])
# simply concatenate both dataframes
us_bus = pd.concat([new_row, us_bus]).reset_index(drop = True)
# us_bus=us_bus.head(20)
us_bus.shape

(198398, 3)

In [8]:
new_row = pd.DataFrame({'User':'u0', 'Users':'u1', 'relationship':1,},index =[0])
us_us = pd.concat([new_row, us_us]).reset_index(drop = True)
# us_us = us_us.head(20)
us_us.shape

(158591, 3)

In [9]:
node_businesses = list(set(us_bus['Business'].to_list()))
len(node_businesses)

14284

In [10]:
node_users = list(set(us_bus['User'].to_list() + us_us['User'].to_list() + us_us['Users'].to_list()))
len(node_users)

16240

In [11]:
# Creating nodes
G_users = nx.Graph()
G_businesses=nx.Graph()
# Add nodes to the graph
for node in node_users:
    G_users.add_node(node)
    
for node in node_businesses:
    G_businesses.add_node(node)

In [12]:
# Setting attribute name and value to nodes created above to distinguish entities
users =  node_users
user_dict = dict(zip(G_users.nodes(), users))
nx.set_node_attributes(G_users, values = user_dict, name='User')

businesses = node_businesses
business_dict = dict(zip(G_businesses.nodes(), businesses))
nx.set_node_attributes(G_businesses, values = business_dict, name='Business')

In [13]:
# Bringing user nodes and businesses nodes into same graph database 
G = nx.compose(G_users,G_businesses)

In [14]:
# Adding relationship "rating"
for index, row in us_bus.iterrows():
    G.add_edge(row['User'], row['Business'], rating=row['Rating'], directed=True)

In [15]:
# Adding relationship "friend"
for index, row in us_us.iterrows():
    G.add_edge(row['User'], row['Users'], friend=row['relationship'])

In [16]:
# Not the output I expected !! WTH,
list(G.nodes(data = True))[::4000]

[('u8452', {'User': 'u8452'}),
 ('u13131', {'User': 'u13131'}),
 ('u7641', {'User': 'u7641'}),
 ('u14137', {'User': 'u14137'}),
 ('u13008', {'User': 'u13008'}),
 ('b1295', {'Business': 'b1295'}),
 ('b375', {'Business': 'b375'}),
 ('b9798', {'Business': 'b9798'})]

In [17]:
list(G.nodes(data=True))[::10000]

[('u8452', {'User': 'u8452'}),
 ('u11844', {'User': 'u11844'}),
 ('b1295', {'Business': 'b1295'}),
 ('b744', {'Business': 'b744'})]

In [18]:
list(G.edges(data = True))[::10000]

[('u8452', 'b3461', {'rating': 2, 'directed': True}),
 ('u13795', 'b8968', {'rating': 4, 'directed': True}),
 ('u14318', 'u10956', {'friend': 1}),
 ('u13266', 'u4549', {'friend': 1}),
 ('u10574', 'b3200', {'rating': 3, 'directed': True}),
 ('u9805', 'u2014', {'friend': 1}),
 ('u4960', 'b2779', {'rating': 5, 'directed': True}),
 ('u1539', 'u11069', {'friend': 1}),
 ('u12023', 'b13174', {'rating': 4, 'directed': True}),
 ('u5049', 'u13750', {'friend': 1}),
 ('u2418', 'b9311', {'rating': 1, 'directed': True}),
 ('u2888', 'u2274', {'friend': 1}),
 ('u538', 'b1701', {'rating': 4, 'directed': True}),
 ('u5630', 'b2756', {'rating': 5, 'directed': True}),
 ('u15624', 'u6228', {'friend': 1}),
 ('u14835', 'b10103', {'rating': 3, 'directed': True}),
 ('u13791', 'b6424', {'rating': 2, 'directed': True}),
 ('u5524', 'b13415', {'rating': 4, 'directed': True}),
 ('u16124', 'b3068', {'rating': 3, 'directed': True}),
 ('u6228', 'b12262', {'rating': 4, 'directed': True}),
 ('u2448', 'b14250', {'rating':

In [19]:
# setting sum and count to zero.
s = 0
c = 0

In [20]:
G.nodes['b8391']['Business']

'b8391'

In [21]:
for u, v, data in G.edges(data=True):
  if data.get('friend', 0) == 1:
    common_list = list(nx.common_neighbors(G, u, v))
    for node in common_list:
      if G.nodes[node].get('Business', 0) != 0:
        s += abs(G.get_edge_data(u, node)['rating'] - G.get_edge_data(v, node)['rating'])
        c += 1

print(s/c)

0.8576135123308066


In [22]:
c

198811

In [23]:
len(list(G.edges))

277694